In [6]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from tqdm import tqdm_notebook
from tqdm import tqdm_notebook
import time

In [7]:
krwordrank_raw_data = pd.read_excel('./data/comp_full_data_210211.xlsx') 

In [8]:
krwordrank_raw_data.head(1)

,Unnamed: 0,내역사업명,title,연구목표,연구내용,기대효과,사업명,부처명,지역구분,6T관련기술분류,국가전략기술,한글키워드,영문키워드,과제관리(전문)기관명
0,0,2020년도 창업성장기술개발사업 '전략형 창업과제(BIG3)' 제1차 시행계획 공고,전략형창업과제,적용 모터 분석 \n 구동부 설계\n 궤도 설계\n 차체설계 및 제작 \n 샤시설계...,적용 모터 분석 \n - 전기자동차 적용 모터 분석 및 최적화 모델 검색\n - 감...,프로토 타입 제작 완료를 통한제품 성능평가\n제품 설계 완성 및 조립도 완성\n샤시...,창업성장기술개발(R&D),중소벤처기업부,경기,위의 미래유망신기술(6T) 103개 세분류에 속하지 않는 기타 연구,스마트 자동차 기술,스마트자동차 전기자동차 전지 다목적 4차산업혁명,Smart car EV E cell multipurpose 4IR,중소기업기술정보진흥원


In [9]:
krwordrank_data = krwordrank_raw_data
del krwordrank_data['Unnamed: 0']
#del krwordrank_data['Unnamed: 0.1']

In [10]:
krwordrank_data.isnull().sum()

내역사업명          0
title          0
연구목표           0
연구내용           0
기대효과           0
사업명            0
부처명            0
지역구분           0
6T관련기술분류       0
국가전략기술         0
한글키워드          0
영문키워드          0
과제관리(전문)기관명    0
dtype: int64

In [12]:
#krwordrank_data.dropna(inplace=True)

## 토큰화
- Okt
- Hannanum
- krwordrank (토큰화가 내재되어 있는 패키지로 바로 키워드 추출)
- Kkma

In [13]:
#불용어 제거

'''============불용어 stopword 정의============'''
stopword=''
f =open("./data/stopwords.txt","r", encoding = 'utf-8')
for line in f:
    stopword += line
    
    
#stop_words = "'한다','II','의','가','이','은','들','는','좀','잘','걍','과','도','을','를','으로','자','에','와','한','하다','이며','만','함','당사','하게','특징','등','및','개발','각','하고','수','있는','통한','처럼','위해','에서','해서','할','이다','여러','가지','시키는','시키지','되는','주로','때문에','있도록','등을','그대로','없이','보다','없도록','간','가지는','따라','하였음','된','1차년도','있어야','될','별','당','따른','하여','이에','의한','현재','입니다','첫째','둘째','셋째','위한','걸쳐','그래서','그리고','그러나','하지만','근데','그러므로','으로써','으로서','겠다','합니다','이를','통해','이외의','본','있습니다','있으며','하였고','있으나','있음','시켜','하기','기존의','이용해','이용하여','됩니다','이루어지고','받아','받고','내고자','바탕으로','이러한'"


In [8]:
''' 반환값 : 과제명(국문), keyword, 부처명 category '''
def get_recommendations(index, cosine_sim):

    # 모든 영화에 대해 해당 영화와의 유사도 구하기
    sim_scores = list(enumerate(cosine_sim[index]))
    
    # 유사도에 따라 영화 정렬
    sim_scores = sorted(sim_scores, key=lambda x:x[1], reverse=True)  #내림차순
    
    # 가장 유사한 영화 10개 받아오기
    sim_scores = sim_scores[1:11]
    # 인덱스 받기
    movie_indices = [i[0] for i in sim_scores]

    #가장 유사한 영화 10개 제목 return
    return movie_indices,krwordrank_data['과제명(국문)'].iloc[movie_indices], krwordrank_data['category'].iloc[movie_indices]
    

### 1. Okt

In [41]:
from ckonlpy.tag import Twitter
twitter = Twitter()

C:\Users\user\anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [61]:
'''
1. 토큰화 후 명사/영어 & 불용어 외 select
2. 길이가 1이하 단어 삭제
3. 중복단어 제거
4. 단어들로만 이어진 문장 형태로 바꿔주기
'''

#토큰화 해주고 난 후 중복 값 제거.
#Twitter

def okt_tokenize(text):
    okt_nouns=[]
    text_pos_list = twitter.pos(text)
    for word, tag in text_pos_list:
        if (tag=='Alpha') and (not word in stopword):
            if len(word)>1:
                okt_nouns.append(word.lower())  # 소문자 변경 추가
        elif (tag=="Noun") and (not word in stopword):
            if len(word)>1:
                okt_nouns.append(word)

    okt_nouns=list(set(okt_nouns))
    
    okt_nouns=' '.join(okt_nouns)

    return okt_nouns


In [62]:
twitter.add_dictionary(['머신러닝','딥러닝','인공지능','빅데이터','Machine Learning','MachineLearning',
                        'Deep Learning','bigdata','의과학','의공학','데이터','라이센서','핸드피스',
                        'hand piece','handpiece','sw','하이브리드','보철물','지르코니아','성형체','우회기법',
                        '고반응성'], 'Noun')


In [63]:
start_time = time.time()
krwordrank_data['연구_okt']=''

i=0
for row in tqdm_notebook(krwordrank_data['연구']): 
    krwordrank_data['연구_okt'].iloc[i]=okt_tokenize(row)
    i+=1
    
end_time = time.time()
print("실행시간 : ", end_time-start_time)
    

<ipython-input-63-7b0dc256b763>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for row in tqdm_notebook(krwordrank_data['연구']):



실행시간 :  207.44569778442383


In [76]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

tfidf = TfidfVectorizer(analyzer = 'word', ngram_range = (1, 2), min_df = 2)
tfidf_matrix = tfidf.fit_transform(twitter_token)

# 코사인 유사도를 통해 문사 유사도 구하기
tw_cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

get_recommendations(0,tw_cosine_sim)

([1],
 1    2019년 창업성장기술개발사업 혁신형 창업과제 혁신성장패키지
 Name: 과제명(국문), dtype: object,
 1    0
 Name: category, dtype: int64)

### 2. Hannanum

In [53]:
from konlpy.tag import Hannanum

hannanum = Hannanum()

In [64]:
'''
1. 토큰화 후 명사/영어 & 불용어 외 select
2. 길이가 1이하 단어 삭제
3. 중복단어 제거
4. 단어들로만 이어진 문장 형태로 바꿔주기
'''

#토큰화 해주고 난 후 중복 값 제거.
#Twitter

def hannanum_tokenize(text):
    han_nouns=[]
    text_pos_list = hannanum.pos(text)
    for word, tag in text_pos_list:
        if (tag in ('F', "N")) and (not word in stopword):
            if len(word)>1:
                han_nouns.append(word)

    han_nouns=list(set(han_nouns))
    
    han_nouns=' '.join(han_nouns)

    return han_nouns


In [65]:
start_time = time.time()
krwordrank_data['연구_hannanum']=''

i=0
try:
    for row in tqdm_notebook(krwordrank_data['연구']): 
        krwordrank_data['연구_hannanum'].iloc[i]=hannanum_tokenize(row)
        i+=1

    end_time = time.time()
    print("실행시간 : ", end_time-start_time)
except:
    print(i)
    pass
    

<ipython-input-65-c083a6dcfaac>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for row in tqdm_notebook(krwordrank_data['연구']):



실행시간 :  434.03483605384827


In [84]:
tfidf = TfidfVectorizer(analyzer = 'word', ngram_range = (1, 2), min_df = 2)
tfidf_matrix = tfidf.fit_transform(hannaum_token)

# 코사인 유사도를 통해 문사 유사도 구하기
hn_cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

get_recommendations(0,hn_cosine_sim)

([1],
 1    2019년 창업성장기술개발사업 혁신형 창업과제 혁신성장패키지
 Name: 과제명(국문), dtype: object,
 1    0
 Name: category, dtype: int64)

### 3. Kkma

In [58]:
from konlpy.tag import Kkma  
kkma=Kkma() 

In [59]:
'''
1. 토큰화 후 명사/영어 & 불용어 외 select
2. 길이가 1이하 단어 삭제
3. 중복단어 제거
4. 단어들로만 이어진 문장 형태로 바꿔주기
'''
def kkma_tokenize(text):
    kkma_nouns=[]
    text_pos_list = kkma.pos(text)
    for word, tag in text_pos_list:
        if (tag in ('OL', "NNG")) and (not word in stopword):
            if len(word)>1:
                kkma_nouns.append(word)

    kkma_nouns=list(set(kkma_nouns))
    
    kkma_nouns=' '.join(kkma_nouns)

    return kkma_nouns


In [60]:
start_time = time.time()
krwordrank_data['연구_kkma']=''

i=0
try:
    for row in tqdm_notebook(krwordrank_data['연구']): 
        krwordrank_data['연구_kkma'].iloc[i]=kkma_tokenize(row)
        i+=1

    end_time = time.time()
    print("Kkma 실행시간 : ", end_time-start_time)
except:
    print(i)
    pass
    

<ipython-input-60-07b409572937>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for row in tqdm_notebook(krwordrank_data['연구']):



Kkma 실행시간 :  4186.323458433151


In [97]:
tfidf = TfidfVectorizer(analyzer = 'word', ngram_range = (1, 2), min_df = 2)
tfidf_matrix = tfidf.fit_transform(kkma_token)

# 코사인 유사도를 통해 문사 유사도 구하기
kk_cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

get_recommendations(0,kk_cosine_sim)

([1],
 1    2019년 창업성장기술개발사업 혁신형 창업과제 혁신성장패키지
 Name: 과제명(국문), dtype: object,
 1    0
 Name: category, dtype: int64)

In [79]:
#토큰화 -- 엑셀파일 저장
writer = pd.ExcelWriter('./project_data/company_token_210208.xlsx', engine='openpyxl')  #경로 각자에 맞게 변경
krwordrank_data.to_excel(writer)
writer.save()